In [1]:
import pandas as pd

In [47]:
#B Small Sample of TriMet data

df = pd.read_csv('bc_trip259172515_230215.csv')
df.head()

,EVENT_NO_TRIP,EVENT_NO_STOP,OPD_DATE,VEHICLE_ID,METERS,ACT_TIME,GPS_LONGITUDE,GPS_LATITUDE,GPS_SATELLITES,GPS_HDOP
0,259172515,259172517,15FEB2023:00:00:00,4223,40,20469,-122.648137,45.493082,12,0.7
1,259172515,259172517,15FEB2023:00:00:00,4223,48,20474,-122.648240,45.493070,12,0.8
2,259172515,259172517,15FEB2023:00:00:00,4223,57,20479,-122.648352,45.493123,12,0.8
3,259172515,259172517,15FEB2023:00:00:00,4223,73,20484,-122.648385,45.493262,12,0.7
4,259172515,259172517,15FEB2023:00:00:00,4223,112,20489,-122.648347,45.493582,12,0.8


In [48]:
#C  Filtering

#filtered_df = df.drop(columns=['EVENT_NO_STOP', 'GPS_SATELLITES', 'GPS_HDOP'])

#filtered_df.head()

In [49]:
#C Filtering using usecols()

columns_to_keep = ['EVENT_NO_TRIP', 'OPD_DATE', 'VEHICLE_ID', 'METERS', 'ACT_TIME', 'GPS_LONGITUDE', 'GPS_LATITUDE']

df_filtered = pd.read_csv('bc_trip259172515_230215.csv', usecols=columns_to_keep)

df_filtered.head()

,EVENT_NO_TRIP,OPD_DATE,VEHICLE_ID,METERS,ACT_TIME,GPS_LONGITUDE,GPS_LATITUDE
0,259172515,15FEB2023:00:00:00,4223,40,20469,-122.648137,45.493082
1,259172515,15FEB2023:00:00:00,4223,48,20474,-122.648240,45.493070
2,259172515,15FEB2023:00:00:00,4223,57,20479,-122.648352,45.493123
3,259172515,15FEB2023:00:00:00,4223,73,20484,-122.648385,45.493262
4,259172515,15FEB2023:00:00:00,4223,112,20489,-122.648347,45.493582


In [50]:
#D Decoding

from datetime import datetime, timedelta

# Define function to create timestamp
def create_timestamp(row):
    opd_date = datetime.strptime(row['OPD_DATE'], '%d%b%Y:%H:%M:%S')
    act_time = timedelta(seconds=row['ACT_TIME'])
    timestamp = opd_date + act_time
    return pd.Timestamp(timestamp)

# Apply the function to create the TIMESTAMP column
df_filtered['TIMESTAMP'] = df_filtered.apply(create_timestamp, axis=1)

df_filtered.head()

,EVENT_NO_TRIP,OPD_DATE,VEHICLE_ID,METERS,ACT_TIME,GPS_LONGITUDE,GPS_LATITUDE,TIMESTAMP
0,259172515,15FEB2023:00:00:00,4223,40,20469,-122.648137,45.493082,2023-02-15 05:41:09
1,259172515,15FEB2023:00:00:00,4223,48,20474,-122.648240,45.493070,2023-02-15 05:41:14
2,259172515,15FEB2023:00:00:00,4223,57,20479,-122.648352,45.493123,2023-02-15 05:41:19
3,259172515,15FEB2023:00:00:00,4223,73,20484,-122.648385,45.493262,2023-02-15 05:41:24
4,259172515,15FEB2023:00:00:00,4223,112,20489,-122.648347,45.493582,2023-02-15 05:41:29


In [51]:
#E More Filtering

df_filtered = df_filtered.drop(columns=['OPD_DATE','ACT_TIME'])

df_filtered.head()

,EVENT_NO_TRIP,VEHICLE_ID,METERS,GPS_LONGITUDE,GPS_LATITUDE,TIMESTAMP
0,259172515,4223,40,-122.648137,45.493082,2023-02-15 05:41:09
1,259172515,4223,48,-122.648240,45.493070,2023-02-15 05:41:14
2,259172515,4223,57,-122.648352,45.493123,2023-02-15 05:41:19
3,259172515,4223,73,-122.648385,45.493262,2023-02-15 05:41:24
4,259172515,4223,112,-122.648347,45.493582,2023-02-15 05:41:29


In [52]:
#F Enhance

# Calculate dMETERS and dTIMESTAMP
df_filtered['dMETERS'] = df_filtered['METERS'].diff()
df_filtered['dTIMESTAMP'] = df_filtered['TIMESTAMP'].diff()

# Calculate SPEED
df_filtered['SPEED'] = df_filtered['dMETERS'] / df_filtered['dTIMESTAMP'].dt.total_seconds()

# Drop unneeded columns
df_filtered.drop(columns=['dMETERS', 'dTIMESTAMP'], inplace=True)

df_filtered.head()

,EVENT_NO_TRIP,VEHICLE_ID,METERS,GPS_LONGITUDE,GPS_LATITUDE,TIMESTAMP,SPEED
0,259172515,4223,40,-122.648137,45.493082,2023-02-15 05:41:09,NaN
1,259172515,4223,48,-122.648240,45.493070,2023-02-15 05:41:14,1.6
2,259172515,4223,57,-122.648352,45.493123,2023-02-15 05:41:19,1.8
3,259172515,4223,73,-122.648385,45.493262,2023-02-15 05:41:24,3.2
4,259172515,4223,112,-122.648347,45.493582,2023-02-15 05:41:29,7.8


In [53]:
# Find statistics
speed_stats = df_filtered['SPEED'].describe()
# Display statistics
print("Minimum Speed:", speed_stats['min'])
print("Maximum Speed:", speed_stats['max'])
print("Average Speed:", speed_stats['mean'])

Minimum Speed: 0.0
Maximum Speed: 17.4
Average Speed: 7.227205815018314
